In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

# 데이터 경로 설정
base_path = "C:/Users/m/Desktop/머신러닝 사용 데이터"
sales_data_path = f"{base_path}/구 데이터/피처엔지니어링한통합데이터/피처엔지니어링일단다한통합데이터.csv"
pop_data_path = f"{base_path}/서울 일별 유동인구.csv"
econ_index_path = f"{base_path}/전처리 데이터/경제심리지수.csv"
cpi_data_path = f"{base_path}/전처리 데이터/소비자물가지수.csv"

# 1. 데이터 로드
sales_df = pd.read_csv(sales_data_path)
pop_df = pd.read_csv(pop_data_path)
econ_df = pd.read_csv(econ_index_path)
cpi_df = pd.read_csv(cpi_data_path)

# 2. 데이터 전처리
# 기준_년분기_코드 변환 (20191 → 2019Q1)
sales_df["기준_년분기"] = sales_df["기준_년분기_코드"].astype(str).apply(lambda x: f"{x[:4]}Q{int(x[4])}")

# 사용자 입력값 받기 (올바른 값이 입력될 때까지 반복)
while True:
    selected_district = input("예측할 상권명을 입력하세요: ").strip()
    selected_industry = input("예측할 업종명을 입력하세요: ").strip()

    # 3. 선택한 상권-업종 데이터 필터링
    filtered_sales = sales_df[(sales_df["상권_코드_명"] == selected_district) &
                              (sales_df["서비스_업종_코드_명"] == selected_industry)]

    if not filtered_sales.empty:
        break  # 유효한 데이터가 있으면 반복 종료
    else:
        print("⚠️ 입력한 상권명 또는 업종명이 데이터에 없습니다. 다시 입력하세요.\n")

# 4. 해당 상권이 속한 자치구 찾기
selected_district_code = filtered_sales["자치구_코드_명"].iloc[0]

# 5. 해당 자치구 전체 매출 데이터 가져오기
district_sales = sales_df[sales_df["자치구_코드_명"] == selected_district_code]

# 6. 분기별 자치구 전체 매출 합산
district_sales_grouped = district_sales.groupby("기준_년분기")["당월_매출_금액"].sum().reset_index()
district_sales_grouped.rename(columns={"당월_매출_금액": "자치구_총매출"}, inplace=True)

# 7. 해당 상권이 차지하는 비율 계산
sales_with_ratio = filtered_sales.merge(district_sales_grouped, on="기준_년분기")
sales_with_ratio["매출_비율"] = sales_with_ratio["당월_매출_금액"] / sales_with_ratio["자치구_총매출"]

# 8. 유동인구 데이터 연결 (자치구 기준)
pop_df.rename(columns={"시군구명": "자치구_코드_명", "총생활인구수": "유동인구"}, inplace=True)
sales_with_ratio = sales_with_ratio.merge(pop_df[["자치구_코드_명", "유동인구"]], on="자치구_코드_명", how="left")

# 9. 경제심리지수 및 소비자물가지수 연결
econ_df["날짜"] = pd.to_datetime(econ_df["날짜"])
cpi_df["날짜"] = pd.to_datetime(cpi_df["날짜"])

econ_df["기준_년분기"] = econ_df["날짜"].dt.to_period("Q").astype(str)
cpi_df["기준_년분기"] = cpi_df["날짜"].dt.to_period("Q").astype(str)

sales_with_ratio = sales_with_ratio.merge(econ_df[["기준_년분기", "경제심리지수"]], on="기준_년분기", how="left")
sales_with_ratio = sales_with_ratio.merge(cpi_df[["기준_년분기", "소비자물가지수"]], on="기준_년분기", how="left")

# 10. 학습용 데이터 준비
features = ["유동인구", "경제심리지수", "소비자물가지수"]
target = "매출_비율"

sales_with_ratio.dropna(inplace=True)  # 결측치 제거
X = sales_with_ratio[features]
y = sales_with_ratio[target]

# 11. 학습/검증 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 12. 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 13. 검증 데이터 평가
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\n📊 모델 검증 결과")
print(f"MAPE (평균 절대 퍼센트 오차): {mape:.4f}")
print(f"RMSE (평균 제곱근 오차): {rmse:.4f}\n")

# 14. 전체 데이터로 다시 학습 후 2024년 3분기 예측
model.fit(X, y)

# 15. 2024년 3분기 예측을 위한 데이터 생성
future_quarter = "2024Q3"
latest_cpi = cpi_df[cpi_df["기준_년분기"] == future_quarter]["소비자물가지수"].values[0]
latest_econ = econ_df[econ_df["기준_년분기"] == future_quarter]["경제심리지수"].values[0]
latest_pop = pop_df[pop_df["자치구_코드_명"] == selected_district_code]["유동인구"].mean()

future_X = pd.DataFrame([[latest_pop, latest_econ, latest_cpi]], columns=features)
predicted_ratio = model.predict(future_X)[0]

# 16. 2024년 3분기 자치구 매출 예측 후 상권별 적용
district_total_future_sales = district_sales_grouped[district_sales_grouped["기준_년분기"] == future_quarter]["자치구_총매출"].values[0]
predicted_sales = district_total_future_sales * predicted_ratio

print(f"📌 선택한 상권: {selected_district}")
print(f"📌 선택한 업종: {selected_industry}")
print(f"📈 예측된 2024년 3분기 매출: {predicted_sales:,.0f} 원")


📊 모델 검증 결과
MAPE (평균 절대 퍼센트 오차): 0.0000
RMSE (평균 제곱근 오차): 0.0000



IndexError: index 0 is out of bounds for axis 0 with size 0

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# 파일 경로 설정
sales_data_path = "C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\구 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv"
pop_data_path = "C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\서울 일별 유동인구.csv"
econ_index_path = "C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\전처리 데이터\\경제심리지수.csv"
cpi_data_path = "C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\전처리 데이터\\소비자물가지수.csv"

# 데이터 로드
sales_df = pd.read_csv(sales_data_path)
pop_df = pd.read_csv(pop_data_path)
econ_df = pd.read_csv(econ_index_path)
cpi_df = pd.read_csv(cpi_data_path)

# 사용자 입력
selected_district = input("자치구를 입력하세요: ")
selected_business = input("업종명을 입력하세요: ")

# 2019~2024년 2분기까지의 데이터 필터링
sales_df = sales_df[sales_df['기준_년분기_코드'] <= 20242]
filtered_sales = sales_df[(sales_df['상권_코드_명'] == selected_district) & 
                           (sales_df['서비스_업종_코드_명'] == selected_business)]

if filtered_sales.empty:
    raise ValueError("선택한 상권-업종 데이터가 없습니다.")

# 자치구 매출 총합 계산
sales_df['기준_년분기'] = sales_df['기준_년분기_코드']
district_sales_grouped = sales_df.groupby(['기준_년분기', '자치구_코드_명'])['당월_매출_금액'].sum().reset_index()
district_sales_grouped.rename(columns={'당월_매출_금액': '자치구_총매출'}, inplace=True)

# 선택한 상권의 자치구 찾기
selected_district_code = filtered_sales["자치구_코드_명"].iloc[0]
district_sales_filtered = district_sales_grouped[district_sales_grouped['자치구_코드_명'] == selected_district_code]

# 매출 비율 계산
sales_ratio = filtered_sales.groupby("기준_년분기")["당월_매출_금액"].sum() / 
district_sales_filtered.groupby("기준_년분기")["자치구_총매출"].sum()
sales_ratio = sales_ratio.fillna(0).reset_index()
sales_ratio.rename(columns={"당월_매출_금액": "매출_비율"}, inplace=True)

# 유동인구, 경제심리지수, 소비자물가지수 결합
econ_df['년월'] = econ_df['날짜'].str[:7]
cpi_df['년월'] = cpi_df['날짜'].str[:7]
pop_df['년월'] = pop_df['시군구명'].map(lambda x: str(x)[:7])
merged_data = sales_ratio.merge(pop_df, left_on='기준_년분기', right_on='년월', how='left')
merged_data = merged_data.merge(econ_df, on='년월', how='left')
merged_data = merged_data.merge(cpi_df, on='년월', how='left')

# 학습 데이터 준비
X = merged_data[['총생활인구수', '경제심리지수', '소비자물가지수']]
y = merged_data['매출_비율']

# 모델 학습
model = LinearRegression()
model.fit(X, y)

# 미래 데이터 예측
future_data = merged_data.iloc[-1].copy()
future_data['기준_년분기'] = 20243
future_X = future_data[['총생활인구수', '경제심리지수', '소비자물가지수']].values.reshape(1, -1)
predicted_ratio = model.predict(future_X)[0]

# 2024년 3분기 자치구 매출 예측 후 상권별 적용
future_quarter = 20243
future_district_sales = district_sales_filtered.iloc[-1]['자치구_총매출'] * 1.02  # 성장률 적용 (가정치)
predicted_sales = future_district_sales * predicted_ratio

print(f"📌 선택한 상권: {selected_district}")
print(f"📌 선택한 업종: {selected_business}")
print(f"📊 예측된 2024년 3분기 매출: {predicted_sales:,.0f} 원")

SyntaxError: invalid syntax (1169470658.py, line 39)

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# 1. 데이터 로드
data_path = "C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\구 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv"
pop_data_path = "C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\서울 일별 유동인구.csv"
econ_index_path = "C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\전처리 데이터\\경제심리지수.csv"
cpi_data_path = "C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\전처리 데이터\\소비자물가지수.csv"

df = pd.read_csv(data_path)
pop_df = pd.read_csv(pop_data_path)
econ_df = pd.read_csv(econ_index_path)
cpi_df = pd.read_csv(cpi_data_path)

# 2. 사용자 입력 받기
selected_district = input("자치구명을 입력하세요: ")
selected_sector = input("업종명을 입력하세요: ")

# 3. 선택한 상권과 업종 데이터 필터링
filtered_sales = df[(df['자치구_코드_명'] == selected_district) & (df['서비스_업종_코드_명'] == selected_sector)]
if filtered_sales.empty:
    raise ValueError("선택한 자치구와 업종 데이터가 없습니다.")

# 4. 자치구 전체 매출 데이터 계산
district_sales_grouped = df[df['자치구_코드_명'] == selected_district].groupby('기준_년분기_코드')['당월_매출_금액'].sum().reset_index()
sector_sales_grouped = filtered_sales.groupby('기준_년분기_코드')['당월_매출_금액'].sum().reset_index()

# 5. 매출 비율 계산 및 학습 데이터 준비
merged_sales = pd.merge(sector_sales_grouped, district_sales_grouped, on='기준_년분기_코드', suffixes=('_업종', '_자치구'))
merged_sales['매출비율'] = merged_sales['당월_매출_금액_업종'] / merged_sales['당월_매출_금액_자치구']

# 6. 외부 데이터 결합 (경제심리지수, 소비자물가지수)
econ_df['날짜'] = econ_df['날짜'].astype(str).str[:6].astype(int)
cpi_df['날짜'] = cpi_df['날짜'].astype(str).str[:6].astype(int)

merged_sales = merged_sales.merge(econ_df, left_on='기준_년분기_코드', right_on='날짜', how='left')
merged_sales = merged_sales.merge(cpi_df, left_on='기준_년분기_코드', right_on='날짜', how='left')
merged_sales.drop(columns=['날짜_x', '날짜_y'], inplace=True)

# 7. 학습 데이터 및 검증 데이터 분할
X = merged_sales[['경제심리지수', '소비자물가지수']]
y = merged_sales['매출비율']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 8. 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 9. 모델 검증
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"검증 결과: MAE={mae:.4f}, R²={r2:.4f}")

# 10. 2024년 3분기 예측 데이터 준비
future_quarter = 20243
future_X = X.iloc[[-1]]  # 가장 최근 데이터를 기반으로 예측
predicted_ratio = model.predict(future_X)[0]

# 11. 2024년 3분기 자치구 예상 매출 계산
latest_district_sales = district_sales_grouped[district_sales_grouped['기준_년분기_코드'] == 20242]['당월_매출_금액'].values[0]
predicted_district_sales = latest_district_sales * 1.02  # 자치구 매출이 2% 성장한다고 가정
predicted_sales = predicted_district_sales * predicted_ratio

print(f"📌 선택한 자치구: {selected_district}, 선택한 업종: {selected_sector}")
print(f"✅ 2024년 3분기 예측 매출: {predicted_sales:,.0f}원")

ValueError: 선택한 자치구와 업종 데이터가 없습니다.

In [9]:
import pandas as pd

# 데이터 파일 로드
sales_df = pd.read_csv(r"C:\Users\m\Desktop\머신러닝 사용 데이터\구 데이터\피처엔지니어링한통합데이터\피처엔지니어링일단다한통합데이터.csv")
population_df = pd.read_csv(r"C:\Users\m\Desktop\머신러닝 사용 데이터\서울 일별 유동인구.csv")
economic_index_df = pd.read_csv(r"C:\Users\m\Desktop\머신러닝 사용 데이터\전처리 데이터\경제심리지수.csv")
cpi_df = pd.read_csv(r"C:\Users\m\Desktop\머신러닝 사용 데이터\전처리 데이터\소비자물가지수.csv")

# 사용자 입력값 받기
selected_district = input("상권명을 입력하세요: ")
selected_industry = input("업종명을 입력하세요: ")

# 1. 선택된 상권-업종의 매출 데이터 필터링
filtered_sales = sales_df[
    (sales_df['상권_코드_명'] == selected_district) & 
    (sales_df['서비스_업종_코드_명'] == selected_industry)
]

# 2. 선택된 상권이 포함된 자치구 구하기
selected_districts = filtered_sales['자치구_코드_명'].unique()

# 3. 해당 자치구와 업종의 매출 총합 구하기
district_sales_grouped = sales_df[
    (sales_df['자치구_코드_명'].isin(selected_districts)) & 
    (sales_df['서비스_업종_코드_명'] == selected_industry)
].groupby('기준_년분기_코드')['당월_매출_금액'].sum().reset_index()

# 4. 선택된 상권-업종의 매출 비율 계산
selected_total_sales = filtered_sales.groupby('기준_년분기_코드')['당월_매출_금액'].sum().reset_index()
merged_sales = selected_total_sales.merge(district_sales_grouped, on='기준_년분기_코드', suffixes=('_상권', '_자치구'))
merged_sales['매출_비율'] = merged_sales['당월_매출_금액_상권'] / merged_sales['당월_매출_금액_자치구']

# 5. 유동인구 데이터 전처리 및 병합
population_df = population_df.rename(columns={'시군구명': '자치구_코드_명'})
population_df = population_df.groupby('자치구_코드_명')['총생활인구수'].mean().reset_index()

# 🔥 수정된 병합 코드: 컬럼명이 다름을 고려하여 `left_on`, `right_on` 사용
district_sales_grouped = district_sales_grouped.merge(
    population_df, left_on='자치구_코드_명', right_on='시군구명', how='left'
)

# 6. 경제심리지수 및 소비자물가지수 병합
economic_index_df['날짜'] = pd.to_datetime(economic_index_df['날짜'])
cpi_df['날짜'] = pd.to_datetime(cpi_df['날짜'])

# 7. 2019~2024년 2분기까지의 데이터 필터링
district_sales_grouped['기준_년분기_코드'] = district_sales_grouped['기준_년분기_코드'].astype(str)
filtered_data = district_sales_grouped[district_sales_grouped['기준_년분기_코드'].str[:4].astype(int) < 20243]

# 8. 머신러닝 모델 학습 준비
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

features = filtered_data[['기준_년분기_코드', '총생활인구수']]
target = filtered_data['당월_매출_금액']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# 9. 모델 학습 및 예측
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# 10. 모델 평가 및 미래 매출 예측
mae = mean_absolute_error(y_test, predictions)
print(f"모델 MAE(평균 절대 오차): {mae}")

# 11. 2024년 3분기 매출 예측
future_quarter = pd.DataFrame({'기준_년분기_코드': [20243], '총생활인구수': [population_df['총생활인구수'].mean()]})
predicted_sales = model.predict(future_quarter)

# 12. 최종 예측 결과 출력
predicted_sales_value = predicted_sales[0] * merged_sales['매출_비율'].mean()
print(f"예측된 2024년 3분기 {selected_district} {selected_industry} 매출: {predicted_sales_value:.2f} 원")

KeyError: '시군구명'

신규 데이터 생성(매출 데이터 추출, 유동인구 데이터와 병합합)

In [13]:
import pandas as pd

# 📌 1. 매출 데이터 불러오기
sales_file = r"C:\Users\m\Desktop\머신러닝 사용 데이터\구 데이터\피처엔지니어링한통합데이터\피처엔지니어링일단다한통합데이터.csv"
sales_df = pd.read_csv(sales_file, encoding="utf-8-sig")

# 필요한 컬럼만 선택
sales_df = sales_df[['기준_년분기_코드', '상권_코드_명', '서비스_업종_코드_명', '당월_매출_금액', '당월_매출_건수', '총_유동인구_수', '자치구_코드_명']]

# 📌 2. 유동인구 데이터 불러오기
pop_file = r"C:\Users\m\Desktop\머신러닝 사용 데이터\서울 일별 유동인구.csv"
pop_df = pd.read_csv(pop_file, encoding="utf-8-sig")

# 기준일ID를 연분기 형식으로 변환 (예: 20180416 -> 20181)
pop_df['기준_년분기_코드'] = (pop_df['기준일ID'] // 10000) * 10 + ((pop_df['기준일ID'] % 10000) // 3000 + 1)

# 일별 데이터를 분기별로 집계
pop_quarterly_df = pop_df.groupby(['기준_년분기_코드', '시군구명'], as_index=False)[
    ['총생활인구수', '일최대인구수', '일최소인구수', '일최대이동인구수', '서울외유입인구수']
].sum()

# 컬럼명 변경 (매출 데이터와 일관성 유지)
pop_quarterly_df = pop_quarterly_df.rename(columns={'시군구명': '자치구_코드_명'})

# 📌 3. 데이터 병합
merged_df = sales_df.merge(pop_quarterly_df, on=['기준_년분기_코드', '자치구_코드_명'], how='left')

# 📌 4. CSV 파일로 저장
output_file = r"C:\Users\m\Desktop\머신러닝 사용 데이터\병합된_매출_유동인구.csv"
merged_df.to_csv(output_file, encoding="utf-8-sig", index=False)

print(f"데이터 병합 완료! 저장된 파일: {output_file}")

데이터 병합 완료! 저장된 파일: C:\Users\m\Desktop\머신러닝 사용 데이터\병합된_매출_유동인구.csv
